<b><u>Observations</b></u>
<p>1. observation</p>
<p>2. observation</p>
<p>3. observation</p>

In [1]:
# Dependencies
import random
import json
import requests
from pprint import pprint
from config import api_key
import pandas as pd
import seaborn as sns
from citipy import citipy

<h1>Generate City List</h1>

In [23]:
# Create a list of 500 random latitudes and longitudes
# Latitudes range from -90 to +90 and longitutes range from -180 to +180
total_cities = 10
latitude_list = [random.randint(-9000,9000)/100 for x in range(0, total_cities)]
longitude_list = [random.randint(-18000,18000)/100 for x in range(0, total_cities)]

# Create a list of cities and their respective country using citipy
city_list = [citipy.nearest_city(latitude_list[x], longitude_list[x]).city_name for x in range(0, total_cities)]
country_list = [citipy.nearest_city(latitude_list[x], longitude_list[x]).country_code for x in range(0, total_cities)]
print(city_list)
print(country_list)

['cape town', 'punta arenas', 'adrar', 'punta arenas', 'rikitea', 'ushuaia', 'balaipungut', 'rikitea', 'cabo san lucas', 'ushuaia']
['za', 'cl', 'dz', 'cl', 'pf', 'ar', 'id', 'pf', 'mx', 'ar']


<h1>Weather API Call</h1>

In [ ]:
# Let's get the JSON for 100 posts sequentially.
url = "http://api.openweathermap.org/data/2.5/forecast?id=524901&APPID="+api_key

In [ ]:
#Weather API call
#weather = requests.get(url).json()

In [5]:
# Make a request for each of the indices
for x in range(total_cities):
    print(f"Making request number: {x}")

    # Get one of the posts
    post_response = requests.get(url)

    # Save post's JSON
    response_json.append(post_response.json())

Making request number: 0 for ID: 97
Making request number: 1 for ID: 71
Making request number: 2 for ID: 83
Making request number: 3 for ID: 56
Making request number: 4 for ID: 2
Making request number: 5 for ID: 53
Making request number: 6 for ID: 67
Making request number: 7 for ID: 79
Making request number: 8 for ID: 35
Making request number: 9 for ID: 55


In [6]:
# Now we have 10 post objects, 
# which we got by making 100 requests to the API.
print(f"We have {len(response_json)} posts!")

We have 10 posts!


<h1>Export Results</h1>